In [ ]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [ ]:
from pyscipopt import Model, quicksum

**Optimierungsmodell für den Kauf und Verkauf von Strom auf dem Strommarkt**

In [ ]:
# Erstellen einer Modellinstanz
scip = Model()

**Indexmenge**

In [ ]:
H = [n for n in range(1, 25)]
R = [r for r in range(0, 25)]
# Erklärung:
# es gibt 26 Risikoabstufungen, höchstes R -> höchstes Risiko, r=0, kein Risiko
#-> kein Risiko + normaler EW, Risikoklasse 1=4% Risiko und EW ergibt sich aus top 96% der Preisprognosen für h, etc.

# Risikowahrscheinlichkeiten
prob_r = {}
for r in R:
    prob_r[r] = 1 - r/25

print(prob_r)

{0: 1.0, 1: 0.96, 2: 0.92, 3: 0.88, 4: 0.84, 5: 0.8, 6: 0.76, 7: 0.72, 8: 0.6799999999999999, 9: 0.64, 10: 0.6, 11: 0.56, 12: 0.52, 13: 0.48, 14: 0.43999999999999995, 15: 0.4, 16: 0.36, 17: 0.31999999999999995, 18: 0.28, 19: 0.24, 20: 0.19999999999999996, 21: 0.16000000000000003, 22: 0.12, 23: 0.07999999999999996, 24: 0.040000000000000036}


**Batterie-Systemspezifikationen**

In [ ]:
fixe_zykluskosten = True

wirkungsgrad_wechselrichter = 0.985
wirkungsgrad_laden = 0.975
round_trip_efficiency = 0.95
entlade_verlust = wirkungsgrad_laden - round_trip_efficiency

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-(entlade_verlust / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter


f_e = wirkungsgrad_systemeingang # Faktor Einkauf
f_v = wirkungsgrad_systemausgang # Faktor Verkauf


nennkapazität = 40 # MWh brutto
lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbestand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
zykluskosten = 1500 # € / zyklus
mwh_zykluskosten = zykluskosten / zyklus # € / MWh

erlaubte_zyklen_pro_tag = 2

a = anfangsbestand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität

c = 0.5 # nennkapazität / h

# Prozentuale penalty, wenn nicht Zuschlag
penalty_anteil = 1
penalty = 1 + penalty_anteil


# Sicherstellen, dass unsere Faktoren für Systemeingang und -ausgang den multiplizierten Wirkungsgraden entspricht
print(wirkungsgrad_systemeingang*wirkungsgrad_systemausgang)
print(wirkungsgrad_wechselrichter*wirkungsgrad_wechselrichter*round_trip_efficiency)

0.92171375
0.92171375


**Vorhersagedaten**

In [ ]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')

# Was wir benötigen: Erwartungswert für jede Risikoklasse zu jeder Stunde h für Einkauf und Verkauf


# Erwartungswerte nach Risikoklasse und Stunde für Einkauf

preis_verkauf_h_r = {}
preis_einkauf_h_r = {}

import math

for h in H:
  # Berechnung von "Standard-EW" für Stunde h
  stundenprognose = prognose[prognose['Stunde'] == h]
  #Sortiere stundenprognose nach dem Wert in Spalte "Strompreis"
  stundenprognose = stundenprognose.sort_values(by='Strompreis')
  # Index zurücksetzen
  stundenprognose = stundenprognose.reset_index(drop=True)
  #print(stundenprognose)
  for r in R:
    # Nehme den Durschnitt von der Spalte "Strompreis" vom Index r bis zum höchsten Index

    preis_verkauf_h_r[(h, r)] = stundenprognose.loc[r:24, 'Strompreis'].mean()
    preis_einkauf_h_r[(h, r)] = stundenprognose.loc[0:24-r, 'Strompreis'].mean()

p_h = {}

#for h in H:
#    stundenprognose = prognose[prognose['Stunde'] == h]
#    checksum = stundenprognose['Wahrscheinlichkeit'].sum()
#    if checksum != 1:
#        print('WARNING: Prognosedaten unvollständig')

#    erwartungswert_strompreis = (stundenprognose['Strompreis'] * stundenprognose['Wahrscheinlichkeit']).sum() / checksum
#    p_h[h] = erwartungswert_strompreis

#print (p_h)

print(preis_verkauf_h_r)
print(preis_einkauf_h_r)


# Für die Berechnung der Penalty möchten wir den Durchschnittspreis der umliegenden Stunden zu jeder Stunde haben
moving_average_h = {}
for h in H:
    stundenprognose = prognose[prognose['Stunde'].isin([h-1,h,h+1])]
    moving_average_h[h] = stundenprognose['Strompreis'].mean()

print(moving_average_h)

{(1, 0): 81.52, (1, 1): 83.54166666666667, (1, 2): 85.21739130434783, (1, 3): 86.77272727272727, (1, 4): 88.23809523809524, (1, 5): 89.65, (1, 6): 91.05263157894737, (1, 7): 92.44444444444444, (1, 8): 93.82352941176471, (1, 9): 95.1875, (1, 10): 96.6, (1, 11): 98.0, (1, 12): 99.46153846153847, (1, 13): 100.91666666666667, (1, 14): 102.45454545454545, (1, 15): 104.1, (1, 16): 105.77777777777777, (1, 17): 107.625, (1, 18): 109.57142857142857, (1, 19): 111.66666666666667, (1, 20): 114.0, (1, 21): 116.75, (1, 22): 120.0, (1, 23): 124.0, (1, 24): 130.0, (2, 0): 72.24, (2, 1): 74.08333333333333, (2, 2): 75.6086956521739, (2, 3): 77.0, (2, 4): 78.33333333333333, (2, 5): 79.6, (2, 6): 80.84210526315789, (2, 7): 82.11111111111111, (2, 8): 83.3529411764706, (2, 9): 84.625, (2, 10): 85.86666666666666, (2, 11): 87.14285714285714, (2, 12): 88.46153846153847, (2, 13): 89.83333333333333, (2, 14): 91.27272727272727, (2, 15): 92.7, (2, 16): 94.22222222222223, (2, 17): 95.875, (2, 18): 97.71428571428571

**Entscheidungsvariablen**

In [ ]:
e_h={}
v_h={}
# Für jede Risikoklasse und jede Stunde eine binäre Variable
bin_h_r={}
# Für jede Risikoklasse und jede Stunde eine Mengen-Variable
e_h_r={}
v_h_r={}

for h in H:
    #e_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}")
    #v_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}")
    #  Für jede Risikoklasse und jede Stunde eine binäre Variable, ob man zu dieser Stunde diese Risikoklasse an Risiko eingehen möchte
    #for r in R:
    #  bin_h_r[(h, r)] = scip.addVar(vtype='B', name=f"r_{h}_{r}")
    #  Für jede Risikoklasse und jede Stunde eine Mengen-Variable, ob man zu dieser Stunde bei dieser Risikoklasse die entsprechende Menge einkaufen möchte
    for r in R:
      e_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}_{r}")
      v_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}_{r}")


print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [e_1_0, v_1_0, e_1_1, v_1_1, e_1_2, v_1_2, e_1_3, v_1_3, e_1_4, v_1_4, e_1_5, v_1_5, e_1_6, v_1_6, e_1_7, v_1_7, e_1_8, v_1_8, e_1_9, v_1_9, e_1_10, v_1_10, e_1_11, v_1_11, e_1_12, v_1_12, e_1_13, v_1_13, e_1_14, v_1_14, e_1_15, v_1_15, e_1_16, v_1_16, e_1_17, v_1_17, e_1_18, v_1_18, e_1_19, v_1_19, e_1_20, v_1_20, e_1_21, v_1_21, e_1_22, v_1_22, e_1_23, v_1_23, e_1_24, v_1_24, e_2_0, v_2_0, e_2_1, v_2_1, e_2_2, v_2_2, e_2_3, v_2_3, e_2_4, v_2_4, e_2_5, v_2_5, e_2_6, v_2_6, e_2_7, v_2_7, e_2_8, v_2_8, e_2_9, v_2_9, e_2_10, v_2_10, e_2_11, v_2_11, e_2_12, v_2_12, e_2_13, v_2_13, e_2_14, v_2_14, e_2_15, v_2_15, e_2_16, v_2_16, e_2_17, v_2_17, e_2_18, v_2_18, e_2_19, v_2_19, e_2_20, v_2_20, e_2_21, v_2_21, e_2_22, v_2_22, e_2_23, v_2_23, e_2_24, v_2_24, e_3_0, v_3_0, e_3_1, v_3_1, e_3_2, v_3_2, e_3_3, v_3_3, e_3_4, v_3_4, e_3_5, v_3_5, e_3_6, v_3_6, e_3_7, v_3_7, e_3_8, v_3_8, e_3_9, v_3_9, e_3_10, v_3_10, e_3_11, v_3_11, e_3_12, v_3_12, e_3_13, v_3_13, e_3_14, v_

# **Zielfunktion**

In [ ]:
gewinn_kauf_verkauf = quicksum(((preis_verkauf_h_r[(h, r)] * v_h_r[(h, r)] * prob_r[r] +             \
                                 moving_average_h[h] * v_h_r[(h, r)] * (1-prob_r[r]) * 1/penalty) -       \

                                (preis_einkauf_h_r[(h, r)] * e_h_r[(h, r)] * prob_r[r] +             \
                                 moving_average_h[h] * e_h_r[(h, r)] * (1-prob_r[r]) * penalty ))        \

                               for r in R for h in H )

if fixe_zykluskosten:
    zykluskosten = 3000
else:
    zykluskosten = quicksum((e_h_r[(h,r)] * f_e * mwh_zykluskosten) for h in H for r in R)

scip.setObjective(gewinn_kauf_verkauf - zykluskosten, sense="maximize")
print(scip.getObjective())

Expr({Term(e_1_0): -81.52, Term(v_1_0): 81.52, Term(e_1_1): -82.4704, Term(v_1_1): 81.7376, Term(e_1_2): -83.90079999999999, Term(v_1_2): 81.4752, Term(e_1_3): -85.57119999999999, Term(v_1_3): 80.97279999999999, Term(e_1_4): -87.4416, Term(v_1_4): 80.27040000000001, Term(e_1_5): -89.472, Term(v_1_5): 79.40800000000002, Term(e_1_6): -91.6224, Term(v_1_6): 78.4256, Term(e_1_7): -93.89280000000001, Term(v_1_7): 77.3232, Term(e_1_8): -96.28320000000001, Term(v_1_8): 76.10079999999999, Term(e_1_9): -98.7936, Term(v_1_9): 74.7584, Term(e_1_10): -101.384, Term(v_1_10): 73.336, Term(e_1_11): -104.09439999999998, Term(v_1_11): 71.7936, Term(e_1_12): -106.88479999999998, Term(v_1_12): 70.1712, Term(e_1_13): -109.7552, Term(v_1_13): 68.4288, Term(e_1_14): -112.74560000000001, Term(v_1_14): 66.6064, Term(e_1_15): -115.81599999999999, Term(v_1_15): 64.704, Term(e_1_16): -119.00639999999999, Term(v_1_16): 62.681599999999996, Term(e_1_17): -122.2768, Term(v_1_17): 60.5792, Term(e_1_18): -125.6672, Te

***Nebenbedingungen/ Restriktionen***

In [ ]:
# Ladestand zur Stunde 0 = Ladestand zur Stunde 24, also Summe Lademenge und Entlademenge gleich
scip.addCons(quicksum(((e_h_r[(h,r)] * f_e) - (v_h_r[(h,r)] / f_v)) for r in R for h in H) == 0, name="Anfangs- und Endbestand gleich")

# Maximale Ladezyklen am pro Tag anhand der Einkaufsmenge (mit Faktor = Lademenge), alternativ anhand der Verkaufsmenge
scip.addCons(quicksum((e_h_r[(h,r)] * f_e) for r in R for h in H) <= (erlaubte_zyklen_pro_tag * nettokapazität), name="Maximale Ladezyklen pro Tag")

# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
for h in H:
    H_t =  [n for n in range(1, h+1)]
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e) - (v_h_r[(t, r)] / f_v)) for r in R for t in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t={h}")
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e) - (v_h_r[(t, r)] / f_v)) for r in R for t in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t={h}")

# Lade- und Entladeleistung begrenzt (C-Rate)
for h in H:
    scip.addCons(quicksum(((e_h_r[(h, r)] * f_e) + (v_h_r[(h, r)] / f_v)) for r in R) <= c * nennkapazität, name=f"Lade-/Entladeleistung der Stunde h={h}")


print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Anfangs- und Endbestand gleich, Maximale Ladezyklen pro Tag, Mindestladestand zum Zeitpunkt t=1, Maximalladestand zum Zeitpunkt t=1, Mindestladestand zum Zeitpunkt t=2, Maximalladestand zum Zeitpunkt t=2, Mindestladestand zum Zeitpunkt t=3, Maximalladestand zum Zeitpunkt t=3, Mindestladestand zum Zeitpunkt t=4, Maximalladestand zum Zeitpunkt t=4, Mindestladestand zum Zeitpunkt t=5, Maximalladestand zum Zeitpunkt t=5, Mindestladestand zum Zeitpunkt t=6, Maximalladestand zum Zeitpunkt t=6, Mindestladestand zum Zeitpunkt t=7, Maximalladestand zum Zeitpunkt t=7, Mindestladestand zum Zeitpunkt t=8, Maximalladestand zum Zeitpunkt t=8, Mindestladestand zum Zeitpunkt t=9, Maximalladestand zum Zeitpunkt t=9, Mindestladestand zum Zeitpunkt t=10, Maximalladestand zum Zeitpunkt t=10, Mindestladestand zum Zeitpunkt t=11, Maximalladestand zum Zeitpunkt t=11, Mindestladestand zum Zeitpunkt t=12, Maximalladestand zum Zeitpunkt t=12, Mindestladestand zum Zeitpunkt t=13, Maximallades

**Berechnung der Lösung**

In [ ]:
from types import NoneType
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
      v_h = 0
      e_h = 0
      risikoklasse_v = None
      risikodict_v = {}
      risikoklasse_e = None
      risikodict_e = {}
      for r in R:
        risikodict_v[r] = scip.getVal(v_h_r[(h, r)])
        risikodict_e[r] = scip.getVal(e_h_r[(h, r)])
        if scip.getVal(v_h_r[(h, r)]) > 0:
          risikoklasse_v = r

        if scip.getVal(e_h_r[(h, r)]) > 0:
          risikoklasse_e = r

        v_h += scip.getVal(v_h_r[(h, r)])
        e_h += scip.getVal(e_h_r[(h, r)])
      print("EINKAUF Stunde", h, " : " , e_h)
      if risikoklasse_e != None: print("EINKAUF Risikoklasse Stunde", h, " : ", risikoklasse_e)
      #print("EINKAUF Risiko: ", risikodict_e)
      print("Verkauf Stunde", h, " : " , v_h)
      if risikoklasse_v != None: print("VERKAUF Risikoklasse Stunde", h, " : ", risikoklasse_v)
      #print("VERKAUF Risiko: ", risikodict_v)
else:
    print('Problem hat keine Lösung')

Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Gewinn) = 1400.7683154295628
EINKAUF Stunde 1  :  0.0
Verkauf Stunde 1  :  0.0
EINKAUF Stunde 2  :  0.0
Verkauf Stunde 2  :  0.0
EINKAUF Stunde 3  :  0.0
Verkauf Stunde 3  :  0.0
EINKAUF Stunde 4  :  20.82519849017311
EINKAUF Risikoklasse Stunde 4  :  0
Verkauf Stunde 4  :  0.0
EINKAUF Stunde 5  :  0.0
Verkauf Stunde 5  :  0.0
EINKAUF Stunde 6  :  0.0
Verkauf Stunde 6  :  0.0
EINKAUF Stunde 7  :  0.0
Verkauf Stunde 7  :  11.516923076923078
VERKAUF Risikoklasse Stunde 7  :  2
EINKAUF Stunde 8  :  0.0
Verkauf Stunde 8  :  19.194871794871794
VERKAUF Risikoklasse Stunde 8  :  3
EINKAUF Stunde 9  :  0.0
Verkauf Stunde 9  :  0.0
EINKAUF Stunde 10  :  0.0
Verkauf Stunde 10  :  0.0
EINKAUF Stunde 11  :  0.0
Verkauf Stunde 11  :  0.0
EINKAUF Stunde 12  :  0.0
Verkauf Stunde 12  :  0.0
EINKAUF Stunde 13  :  0.0
Verkauf Stunde 13  :  0.0
EINKAUF Stunde 14  :  12.495119094103867
EINKAUF Risikoklasse Stunde 14  :  8
Verkauf Stunde 14  :  0.0


In [ ]:
# Ergebnisse in Excel speichern

import pandas as pd
from openpyxl import load_workbook

# File and worksheet details
file_path = "Ergebnisse.xlsx"
if fixe_zykluskosten:
    sheet_name = "org_fix"
else:
    sheet_name = "org_var"

# Extract solution data from the solved SCIP model
solution_data = {
    "Hour": [h for h in H],
    "Einkauf": [round(scip.getVal(e_h[h]), 3) for h in H],
    "Verkauf": [round(scip.getVal(v_h[h]), 3) for h in H],
    "Gebotspreis": [
    "min" if round(scip.getVal(v_h[h]), 3) > 0 else "max" if round(scip.getVal(e_h[h]), 3) > 0 else "" 
    for h in H]
}

solution_df = pd.DataFrame(solution_data)

# Write the solution to the Excel file
try:
    # Load the workbook to check for existing worksheets
    workbook = load_workbook(file_path)
    if sheet_name not in workbook.sheetnames:
        # If the worksheet doesn't exist, create it
        print(f"Worksheet '{sheet_name}' not found. Creating it...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        # If the worksheet exists, overwrite it
        print(f"Worksheet '{sheet_name}' found. Writing solution...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file doesn't exist, create it and write the solution
    print(f"File '{file_path}' not found. Creating it and writing solution...")
    with pd.ExcelWriter(file_path, mode="w", engine="openpyxl") as writer:
        solution_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Solution written to '{file_path}' in the '{sheet_name}' worksheet.")
